## Imports

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from path_explain import utils
utils.set_up_environment(visible_devices='3')

In [4]:
import tensorflow as tf
import tensorflow_datasets
import numpy as np
import scipy
from transformers import *
from plot.text import text_plot

## Data and Model Loading

In [5]:
task = 'sst-2'
num_labels = len(glue_processors[task]().get_labels())

In [10]:
config = DistilBertConfig.from_pretrained('.', num_labels=num_labels)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('.', config=config)

In [11]:
data, info = tensorflow_datasets.load('glue/sst2', with_info=True)

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset glue (/homes/gws/psturm/tensorflow_datasets/glue/sst2/0.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from /homes/gws/psturm/tensorflow_datasets/glue/sst2/0.0.2


In [12]:
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task=task)
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task=task)
valid_dataset = valid_dataset.batch(16)

## Model Evaluation

In [13]:
valid_pred = model.predict(valid_dataset)

In [14]:
valid_input = []
valid_labels = []
for batch in valid_dataset:
    valid_input.append(batch[0])
    valid_labels.append(batch[1].numpy())
valid_labels_np = np.concatenate(valid_labels, axis=0)

In [15]:
valid_pred_max = np.argmax(valid_pred, axis=-1)
accuracy = np.sum(valid_pred_max == valid_labels_np) / len(valid_labels_np)

positive_mask = valid_labels_np == 1
positive_accuracy = np.sum(valid_pred_max[positive_mask] == valid_labels_np[positive_mask]) / np.sum(positive_mask)

negative_mask = valid_labels_np == 0
negative_accuracy = np.sum(valid_pred_max[negative_mask] == valid_labels_np[negative_mask]) / np.sum(negative_mask)

print('Validation Accuracy: {:.4f}'.format(accuracy))
print('Positive Sentiment Accuracy: {:.4f}'.format(positive_accuracy))
print('Negative Sentiment Accuracy: {:.4f}'.format(negative_accuracy))

Validation Accuracy: 0.9071
Positive Sentiment Accuracy: 0.9054
Negative Sentiment Accuracy: 0.9089


In [16]:
for item in valid_dataset.take(1):
    print(item)

({'input_ids': <tf.Tensor: shape=(16, 128), dtype=int32, numpy=
array([[  101,  2009, 23283, ...,     0,     0,     0],
       [  101,  2205,  2172, ...,     0,     0,     0],
       [  101,  1037,  2307, ...,     0,     0,     0],
       ...,
       [  101,  2004,  1996, ...,     0,     0,     0],
       [  101,  1996,  2516, ...,     0,     0,     0],
       [  101,  9327,  1012, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(16, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(16, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [20]:
model(item[0])[0]

<tf.Tensor: shape=(16, 2), dtype=float32, numpy=
array([[-3.9101574 ,  3.8021853 ],
       [ 3.8041704 , -2.7883568 ],
       [ 1.9888983 , -1.4849629 ],
       [ 3.4512417 , -2.530995  ],
       [-2.443832  ,  2.26392   ],
       [-4.1048136 ,  4.016287  ],
       [-1.5832212 ,  1.3562965 ],
       [ 2.9390903 , -2.1744635 ],
       [-0.35801238,  0.43716034],
       [-4.2704735 ,  4.176792  ],
       [-3.441905  ,  3.28892   ],
       [-2.6926115 ,  2.5206203 ],
       [ 2.647887  , -1.9844501 ],
       [-4.2204533 ,  4.191584  ],
       [ 0.8310715 , -0.6177254 ],
       [-4.196546  ,  4.136347  ]], dtype=float32)>

In [45]:
input_ids = item[0]['input_ids']
embeddings = model.distilbert.embeddings(input_ids)

In [46]:
# attention_mask = tf.ones(input_ids.shape)
# attention_mask = tf.cast(attention_mask, dtype=tf.float32)
attention_mask = tf.cast(input_ids != 0, dtype=tf.float32)
head_mask = [None] * model.distilbert.num_hidden_layers

transformer_output = model.distilbert.transformer([embeddings, attention_mask, head_mask], training=False)[0]

In [47]:
pooled_output = transformer_output[:, 0]
pooled_output = model.pre_classifier(pooled_output)
logits = model.classifier(pooled_output)

In [48]:
logits

<tf.Tensor: shape=(16, 2), dtype=float32, numpy=
array([[-3.9101574 ,  3.8021853 ],
       [ 3.8041704 , -2.7883568 ],
       [ 1.9888983 , -1.4849629 ],
       [ 3.4512417 , -2.530995  ],
       [-2.443832  ,  2.26392   ],
       [-4.1048136 ,  4.016287  ],
       [-1.5832212 ,  1.3562965 ],
       [ 2.9390903 , -2.1744635 ],
       [-0.35801238,  0.43716034],
       [-4.2704735 ,  4.176792  ],
       [-3.441905  ,  3.28892   ],
       [-2.6926115 ,  2.5206203 ],
       [ 2.647887  , -1.9844501 ],
       [-4.2204533 ,  4.191584  ],
       [ 0.8310715 , -0.6177254 ],
       [-4.196546  ,  4.136347  ]], dtype=float32)>